In [ ]:
from scipy.optimize import minimize
import pandas as pd
import numpy as np
from datetime import date
from sklearn.linear_model import LassoCV
from sklearn.model_selection import KFold
from sklearn.feature_selection import mutual_info_regression
from sklearn.linear_model import LassoCV, BayesianRidge, LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from scipy.stats import multivariate_normal
import sys
import os
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.model_selection import GridSearchCV
import scipy.stats as ss
from scipy import stats
from scipy.stats import spearmanr
from scipy import interpolate
from sklearn.covariance import GraphicalLassoCV, ledoit_wolf
from sklearn.preprocessing import *
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import GaussianNB
import matplotlib.pyplot as plt


np.set_printoptions(threshold=sys.maxsize)


class Featureless:
    def fit(self, X, y):
        self.mean = np.mean(y)

    def predict(self, X):
        test_features = X
        test_nrow, test_ncol = test_features.shape
        return np.repeat(self.mean, test_nrow)


class SpearmanRankRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, threshold=0.0):
        self.threshold = threshold
        self.preprocessor1 = make_pipeline(
            # FunctionTransformer(np.log1p),
            # PowerTransformer(),
            StandardScaler(with_std=True),
            # FunctionTransformer(stats.boxcox),
            # LambdaTransformer(func=lambda x: ss.rankdata(x, axis=0)),
            # MinMaxScaler(),
        )

    def fit(self, X, y):
        self.y_train = y
        # X_train_ranked_transf = ss.rankdata(X, axis=0)  
        # self.y_train_ranked_transf = self.preprocessor2.fit_transform(ss.rankdata(y).reshape(-1, 1)).flatten()
        self.y_train_ranked_transf = ss.rankdata(y)
        X_train_ranked_transf = self.preprocessor1.fit_transform(ss.rankdata(X, axis=0),ss.rankdata(y) )
        # X_train_ranked_transf = PowerTransformer().fit_transform(ss.rankdata(X, axis=0))
        # self.y_train_ranked_transf = PowerTransformer().fit_transform(ss.rankdata(y))

        slope_list = []
        intercept_list = []

        for index_col in range(X_train_ranked_transf.shape[1]):
            X_col = X_train_ranked_transf[:, index_col]
            calc_slope, calc_intercept = self.find_model_params(
                X_col, self.y_train_ranked_transf)
            slope_list.append(calc_slope)
            intercept_list.append(calc_intercept)
        # Find the mean of the gradients and intercepts
        self.slope_list = slope_list
        self.intercept_list = intercept_list
        return self

    def find_model_params(self, X_col, y_col):
        calc_cor = np.corrcoef(X_col, y_col)[0, 1]
        # If the correlation is greater than the threshold, then calculate the gradient and intercept
        if abs(calc_cor) > self.threshold:
            calc_slope = calc_cor * np.std(y_col) / np.std(X_col)
            calc_intercept = np.mean(y_col) - calc_slope * np.mean(X_col)
        else:
            calc_slope = None
            calc_intercept = None
        return calc_slope, calc_intercept

    def predict(self, X):
        pred_y_list = []
        X_test_ranked_transf = self.preprocessor1.fit_transform(ss.rankdata(X, axis=0))
        # X_test_ranked_transf = ss.rankdata(X, axis=0)

        for index_col in range(X_test_ranked_transf.shape[1]):
            X_col = X_test_ranked_transf[:, index_col]
            # use the average of the slope_list as the default slope
            filtered_slope_list = [x for x in self.slope_list if x is not None]
            mean_filtered_slope = np.mean(filtered_slope_list) if len(
                filtered_slope_list) > 0 else 0
            calc_slope = mean_filtered_slope if self.slope_list[
                index_col] is None else self.slope_list[index_col]

            filtered_intercept_list = [
                x for x in self.intercept_list if x is not None]
            mean_filtered_intercept = np.mean(filtered_intercept_list) if len(
                filtered_intercept_list) > 0 else 0
            calc_intercept = mean_filtered_intercept if self.intercept_list[
                index_col] is None else self.intercept_list[index_col]

            calc_y_ranked = calc_slope * X_col + calc_intercept

            # remove duplicate values from self.y_train_ranked_transf and use indexes to remove items from self.y_train
            y_train_ranked_transf_unique, sorted_indexes = np.unique(self.y_train_ranked_transf, return_index=True)
            y_train_unique = self.y_train[sorted_indexes]

            linear_interpolation = interpolate.interp1d(y_train_ranked_transf_unique, y_train_unique, fill_value="extrapolate")

            # cubic_spline_interpolation = interpolate.CubicSpline(
            #     y_train_ranked_transf_unique, y_train_unique,  extrapolate=True)
            calc_y = linear_interpolation(calc_y_ranked)

            pred_y_list.append(calc_y)
        # Find the mean of the predicted y values
        pred_y = np.mean(np.array(pred_y_list), axis=0)
        return pred_y



data_set_name = "amgut1"
n_of_samples = 50
index_of_pred_col = 2

# Name some string contants
out_dir = "/scratch/da2343/cs685fall22/data"
out_file = out_dir + f'/my_algos_{str(date.today())}_results.csv'

dataset_dict = {
    "amgut1": "/home/da2343/cs685_fall22/data/amgut1_data_standard_scaled.csv",
    # "crohns": "/home/da2343/cs685_fall22/data/crohns_data_update.csv",
    # "baxter_crc": "/home/da2343/cs685_fall22/data/baxter_crc_data_power_transformed.csv",
    # "amgut2": "/home/da2343/cs685_fall22/data/amgut2_data_log_standard_scaled_transformed.csv"
}

dataset_path = dataset_dict[data_set_name]
n_splits = 3

# Import the csv file of the dataset
dataset_pd = pd.read_csv(dataset_path, header=0)
sub_data_dict = {}
# drop only one column per every iteration to form the input matrix
# make the column you removed the output
# print the size of the input matrix
output_vec = dataset_pd.iloc[:, index_of_pred_col].to_frame()
input_mat = dataset_pd.drop(dataset_pd.columns[index_of_pred_col], axis=1)

input_mat_update = input_mat.iloc[:n_of_samples].to_numpy()
output_vec_update = output_vec.iloc[:n_of_samples].to_numpy().ravel()

# rank input matrix and output vector
# input_mat_update_ranked = ss.rankdata(input_mat_update, axis=0)
# output_vec_update_ranked = ss.rankdata(output_vec_update)

# # transform input matrix and output vector using PowerTransformer
# input_mat_update_ranked_transformed = PowerTransformer().fit_transform(input_mat_update_ranked)
# output_vec_update_ranked_transformed = PowerTransformer().fit_transform(output_vec_update_ranked.reshape(-1, 1)).ravel()


# input_mat_update = np.sqrt(input_mat_update)
# output_vec_update = np.sqrt(output_vec_update)

data_tuple = (input_mat_update,
              output_vec_update, index_of_pred_col)

# Create a list of alphas for the LASSOCV to cross-validate against
threshold_param_list = np.concatenate(
    (np.linspace(0, 0.2, 125), np.linspace(0.21, 0.4, 21), np.arange(0.5, 1.01, 0.1)))
threshold_param_dict = [{'threshold': [threshold]}
                        for threshold in threshold_param_list]

learner_dict = {
    # 'MultiVariateNormalModel' : MultiVariateNormalModel(),
    # 'GuassianGraphicalMethod' : GuassianGraphicalMethod(),
    # 'Pearson Correlation':  MyPearsonRegressor(),
    'SpearmanRankRegressor': SpearmanRankRegressor(),
    'Featureless': Featureless(),
}


test_err_list = []

(input_mat, output_vec, index_col) = data_tuple
k_fold = KFold(n_splits=n_splits, shuffle=True, random_state=1)
for fold_id, indices in enumerate(k_fold.split(input_mat)):
    index_dict = dict(zip(["train", "test"], indices))
    set_data_dict = {}
    for set_name, index_vec in index_dict.items():
        set_data_dict[set_name] = {
            "X": input_mat[index_vec],
            "y": output_vec[index_vec]
        }
   
    actual_y = set_data_dict["test"]["y"]
    x_col = set_data_dict["test"]["X"][:, index_col]
    
    # Spearman model prediction   
    spearman =  SpearmanRankRegressor()
    spearman.fit(**set_data_dict["train"])
    spearman_pred_y = spearman.predict(set_data_dict["test"]["X"])
    # Featureless model prediction
    featureless = Featureless()
    featureless.fit(**set_data_dict["train"])
    featureless_pred_y = featureless.predict(set_data_dict["test"]["X"])
    

    
    # Graph of the pred_y for both spearman_pred_y and featureless_pred_y and actual_y on the y axis, and the x_col on the x axis
    plt.scatter(x_col, actual_y, label="actual_y")
    plt.scatter(x_col, spearman_pred_y, label="spearman_pred_y")
    plt.scatter(x_col, featureless_pred_y, label="featureless_pred_y")
    plt.legend()
    plt.show()
    
    test_err_list.append({
        "fold_id": fold_id,
        "spearman_test_err2": mean_squared_error(actual_y, spearman_pred_y),
        "featureless_test_err2": mean_squared_error(actual_y, featureless_pred_y),
    })
    

test_err_df = pd.DataFrame(test_err_list)
print(test_err_df)  

In [7]:
from scipy.optimize import minimize
import pandas as pd
import numpy as np
from datetime import date
from sklearn.linear_model import LassoCV
from sklearn.model_selection import KFold
from sklearn.feature_selection import mutual_info_regression
from sklearn.linear_model import LassoCV, BayesianRidge, LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from scipy.stats import multivariate_normal
import sys
import os
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.model_selection import GridSearchCV
import scipy.stats as ss
from scipy import stats
from scipy.stats import spearmanr
from scipy import interpolate
from sklearn.covariance import GraphicalLassoCV, ledoit_wolf
from sklearn.preprocessing import *
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import GaussianNB
import matplotlib.pyplot as plt


np.set_printoptions(threshold=sys.maxsize)


class Featureless:
    def fit(self, X, y):
        self.mean = np.mean(y)

    def predict(self, X):
        test_features = X
        test_nrow, test_ncol = test_features.shape
        return np.repeat(self.mean, test_nrow)


class SpearmanRankRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, threshold=0.0):
        self.threshold = threshold
        self.preprocessor1 = make_pipeline(
            MinMaxScaler(),
            StandardScaler(),
        )
        self.preprocessor2 = make_pipeline(
            MinMaxScaler(),
            StandardScaler(),
        )

    def fit(self, X, y):
        self.y_train = y
        # X_train_ranked_transf = ss.rankdata(X, axis=0)
        self.y_train_ranked_transf = self.preprocessor2.fit_transform(
            ss.rankdata(y).reshape(-1, 1)).flatten()
        # self.y_train_ranked_transf = ss.rankdata(y)
        X_train_ranked_transf = self.preprocessor1.fit_transform(
            ss.rankdata(X, axis=0))
        # X_train_ranked_transf = PowerTransformer().fit_transform(ss.rankdata(X, axis=0))
        # self.y_train_ranked_transf = PowerTransformer().fit_transform(ss.rankdata(y))

        slope_list = []
        intercept_list = []

        for index_col in range(X_train_ranked_transf.shape[1]):
            X_col = X_train_ranked_transf[:, index_col]
            calc_slope, calc_intercept = self.find_model_params(
                X_col, self.y_train_ranked_transf)
            slope_list.append(calc_slope)
            intercept_list.append(calc_intercept)
        # Find the mean of the gradients and intercepts
        self.slope_list = slope_list
        self.intercept_list = intercept_list
        return self

    def find_model_params(self, X_col, y_col):
        calc_cor = np.corrcoef(X_col, y_col)[0, 1]
        # If the correlation is greater than the threshold, then calculate the gradient and intercept
        if abs(calc_cor) > self.threshold:
            calc_slope = calc_cor * np.std(y_col) / np.std(X_col)
            calc_intercept = np.mean(y_col) - calc_slope * np.mean(X_col)
        else:
            calc_slope = None
            calc_intercept = None
        return calc_slope, calc_intercept

    def predict(self, X):
        pred_y_list = []
        X_test_ranked_transf = self.preprocessor1.fit_transform(
            ss.rankdata(X, axis=0))
        # X_test_ranked_transf = ss.rankdata(X, axis=0)

        for index_col in range(X_test_ranked_transf.shape[1]):
            X_col = X_test_ranked_transf[:, index_col]
            # use the average of the slope_list as the default slope
            filtered_slope_list = [x for x in self.slope_list if x is not None]
            mean_filtered_slope = np.mean(filtered_slope_list) if len(
                filtered_slope_list) > 0 else 0
            calc_slope = mean_filtered_slope if self.slope_list[
                index_col] is None else self.slope_list[index_col]

            filtered_intercept_list = [
                x for x in self.intercept_list if x is not None]
            mean_filtered_intercept = np.mean(filtered_intercept_list) if len(
                filtered_intercept_list) > 0 else 0
            calc_intercept = mean_filtered_intercept if self.intercept_list[
                index_col] is None else self.intercept_list[index_col]

            calc_y_ranked = calc_slope * X_col + calc_intercept

            # remove duplicate values from self.y_train_ranked_transf and use indexes to remove items from self.y_train
            y_train_ranked_transf_unique, sorted_indexes = np.unique(
                self.y_train_ranked_transf, return_index=True)
            y_train_unique = self.y_train[sorted_indexes]

            try:
                linear_interpolation = interpolate.interp1d(
                    y_train_ranked_transf_unique, y_train_unique, fill_value="extrapolate")
                calc_y = linear_interpolation(calc_y_ranked)
            except:
                calc_y = np.mean(self.y_train)  
            
            pred_y_list.append(calc_y)
        # Find the mean of the predicted y values
        pred_y = np.mean(np.array(pred_y_list), axis=0)
        return pred_y



class MyPearsonRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, threshold=0.0):
        self.threshold = threshold

    def fit(self, X, y):
        slope_list = []
        intercept_list = []
        for index_col in range(X.shape[1]):
            X_col = X[:, index_col]
            calc_slope, calc_intercept = self.find_model_params(X_col, y)
            slope_list.append(calc_slope)
            intercept_list.append(calc_intercept)
        # Find the mean of the gradients and intercepts
        self.slope_list = slope_list
        self.intercept_list = intercept_list
        return self

    def find_model_params(self, X_col, y_col):
        calc_cor = np.corrcoef(X_col, y_col)[0, 1]
        # If the correlation is greater than the threshold, then calculate the gradient and intercept
        if abs(calc_cor) > self.threshold:
            calc_slope = calc_cor * np.std(y_col) / np.std(X_col)
            calc_intercept = np.mean(y_col) - calc_slope * np.mean(X_col)
        else:
            calc_slope = None
            calc_intercept = None
        return calc_slope, calc_intercept

    def predict(self, X):
        pred_y_list = []
        for index_col in range(X.shape[1]):
            X_col = X[:, index_col]
            # use the average of the slope_list as the default slope
            filtered_slope_list = [x for x in self.slope_list if x is not None]
            mean_filtered_slope = np.mean(filtered_slope_list) if len(
                filtered_slope_list) > 0 else 0
            calc_slope = mean_filtered_slope if self.slope_list[
                index_col] is None else self.slope_list[index_col]

            filtered_intercept_list = [
                x for x in self.intercept_list if x is not None]
            mean_filtered_intercept = np.mean(filtered_intercept_list) if len(
                filtered_intercept_list) > 0 else 0
            calc_intercept = mean_filtered_intercept if self.intercept_list[
                index_col] is None else self.intercept_list[index_col]

            calc_y = calc_slope * X_col + calc_intercept
            pred_y_list.append(calc_y)
        # Find the mean of the predicted y values
        pred_y = np.mean(pred_y_list, axis=0)
        return pred_y



data_set_name = "amgut1"
n_of_samples = 289
index_of_pred_col = 0

# Name some string contants
out_dir = "/scratch/da2343/cs685fall22/data"
out_file = out_dir + f'/my_algos_{str(date.today())}_results.csv'

dataset_dict = {
    "amgut1": "/home/da2343/cs685_fall22/data/amgut1_data_standard_scaled.csv",
    # "crohns": "/home/da2343/cs685_fall22/data/crohns_data_update.csv",
    # "baxter_crc": "/home/da2343/cs685_fall22/data/baxter_crc_data_power_transformed.csv",
    # "amgut2": "/home/da2343/cs685_fall22/data/amgut2_data_log_standard_scaled_transformed.csv"
}

dataset_path = dataset_dict[data_set_name]
n_splits = 3

# Import the csv file of the dataset
dataset_pd = pd.read_csv(dataset_path, header=0)
sub_data_dict = {}
output_vec = dataset_pd.iloc[:, index_of_pred_col].to_frame()
input_mat = dataset_pd.drop(dataset_pd.columns[index_of_pred_col], axis=1)

input_mat_update = input_mat.iloc[:n_of_samples].to_numpy()
output_vec_update = output_vec.iloc[:n_of_samples].to_numpy().ravel()

data_tuple = (input_mat_update,
              output_vec_update, index_of_pred_col)

# Create a list of alphas for the LASSOCV to cross-validate against
# threshold_param_list = np.concatenate(
#     (np.linspace(0, 0.2, 125), np.linspace(0.21, 0.4, 21), np.arange(0.5, 1.01, 0.1)))
# threshold_param_dict = [{'threshold': [threshold]}
#                         for threshold in threshold_param_list]

threshold_param_list = np.concatenate(
    (np.linspace(0, 0.4, 5), np.linspace(0.41, 0.6, 21), np.arange(0.7, 1.01, 0.1)))
threshold_param_dict = [{'threshold': [threshold]}
                        for threshold in threshold_param_list]

learner_dict = {
    # 'MultiVariateNormalModel' : MultiVariateNormalModel(),
    # 'GuassianGraphicalMethod' : GuassianGraphicalMethod(),
    # 'Pearson Correlation':  MyPearsonRegressor(),
    'SpearmanRankRegressor': GridSearchCV(SpearmanRankRegressor(),
                                threshold_param_dict,
                                scoring='neg_mean_squared_error',
                                return_train_score=True),
    'Featureless': Featureless(),
}


test_err_list = []

(input_mat, output_vec, index_col) = data_tuple
k_fold = KFold(n_splits=n_splits, shuffle=True, random_state=1)
for fold_id, indices in enumerate(k_fold.split(input_mat)):
    index_dict = dict(zip(["train", "test"], indices))
    set_data_dict = {}
    for set_name, index_vec in index_dict.items():
        set_data_dict[set_name] = {
            "X": input_mat[index_vec],
            "y": output_vec[index_vec]
        }
   
    actual_y = set_data_dict["test"]["y"]
    x_col = set_data_dict["test"]["X"][:, index_col]
    
    # Pearson model prediction
    pearson = MyPearsonRegressor()
    pearson.fit(**set_data_dict["train"])
    pearson_pred_y = pearson.predict(set_data_dict["test"]["X"])
    # Spearman model prediction   
    spearman = GridSearchCV(SpearmanRankRegressor(),
                                threshold_param_dict, 
                                return_train_score=True,
                                )
    spearman.fit(**set_data_dict["train"])
    # spearman.fit(**set_data_dict["train"])
    spearman_pred_y = spearman.best_estimator_.predict(set_data_dict["test"]["X"])
    # Featureless model prediction
    featureless = Featureless()
    featureless.fit(**set_data_dict["train"])
    featureless_pred_y = featureless.predict(set_data_dict["test"]["X"])
    
    test_err_list.append({
        "fold_id": fold_id,
        "featureless_test_err2": mean_squared_error(actual_y, featureless_pred_y),
        "spearman_test_err2": mean_squared_error(actual_y, spearman_pred_y),
        "pearson_test_err2": mean_squared_error(actual_y, pearson_pred_y),
    })
    

test_err_df = pd.DataFrame(test_err_list)
print(test_err_df)  

   fold_id  featureless_test_err2  spearman_test_err2  pearson_test_err2
0        0               0.322297            0.315993           0.291849
1        1               0.548365            0.662198           0.508963
2        2               2.197654            2.389734           2.127140


In [ ]:
   fold_id  featureless_test_err2  spearman_test_err2  pearson_test_err2
0        0               0.174987            0.137678           0.206267
1        1               0.197332            0.185014           0.205588
2        2               1.835439            1.846116           1.838585